See [wavelength_ranges.ipynb](wavelength_ranges.ipynb) for a derivation of the wavelength ranges used here.

In [1]:
import re
import collections
from pathlib import Path

import multiprocess
from tqdm.notebook import tqdm

import numpy as np

from astropy.io import fits
from astropy import units as u
from astropy import table

from specutils import Spectrum1D
from specutils.manipulation import FluxConservingResampler

In [2]:
wl_bins = np.arange(9000., 20000., .23) << u.angstrom
len(wl_bins)

47827

In [3]:
phoenix_data = Path('phoenix/subgrid')
while not phoenix_data.is_dir():
    phoenix_data = phoenix_data.parent
assert phoenix_data != Path(), 'no part of this path is present!'
phoenix_data

PosixPath('phoenix')

In [4]:
wavepth = list(phoenix_data.glob('*WAVE*'))
assert len(wavepth) == 1
wavepth = wavepth[0]

In [5]:
pattern = re.compile('lte(\d{5})([+-][0-9.]{4})([+-][0-9.]{3}).PHOENIX-ACES-AGSS-COND-2011-HiRes.fits')

PhoenixFiles = collections.namedtuple('PhoenixFiles', 'pth, teff, logg, fehish'.split(', '))

matches = []
for p in phoenix_data.iterdir():
    if m := pattern.match(p.name):
        matches.append(PhoenixFiles(p, int(m.group(1)), float(m.group(2)), float(m.group(3))))
    
matches[0]

PhoenixFiles(pth=PosixPath('phoenix/lte08400-4.50-0.5.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits'), teff=8400, logg=-4.5, fehish=-0.5)

In [6]:
with fits.open(wavepth) as f:
    phoenix_wave = f[0].data << u.Unit(f[0].header['UNIT'])
phoenix_wave

<Quantity [  500.  ,   500.1 ,   500.2 , ..., 54999.25, 54999.5 , 54999.75] Angstrom>

# Some timing tests to see what's feasible

In [7]:
%%timeit
with fits.open(matches[42][0]) as f:
    phoenix_spec = Spectrum1D(spectral_axis=phoenix_wave, flux=f[0].data*u.Unit(f[0].header['BUNIT']))

28.3 ms ± 386 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


There may be some caching there but it seems on the order of 20 ms.

In [8]:
with fits.open(matches[42][0]) as f:
    phoenix_spec = Spectrum1D(spectral_axis=phoenix_wave, flux=f[0].data*u.Unit(f[0].header['BUNIT']))

In [9]:
_resampler= FluxConservingResampler()
def resample_to_bins(wlbins, model_spec, resampler_func=_resampler.resample1d):
    msk = (wlbins[0] <= model_spec.spectral_axis) & (model_spec.spectral_axis<=wlbins[-1])

    phoenix_sub_spec = Spectrum1D(spectral_axis=model_spec.spectral_axis[msk], 
                                            flux=model_spec.flux[msk])

    return resampler_func(phoenix_sub_spec, (wlbins[:-1] + wlbins[1:])/2)

In [10]:
%%timeit

phoenix_lowres = resample_to_bins(wl_bins, phoenix_spec)
phoenix_lowres.write('test.out', format='tabular-fits', overwrite=True)

3.93 s ± 76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
!rm test.out

In [12]:
3.9*len(matches)/60 # min

491.335

Worth parallelizing...

# Resample the model spectra

In [13]:
resampler = FluxConservingResampler()
wl_mid = (wl_bins[:-1] + wl_bins[1:])/2
def do_resample(in_path, overwrite=False, prefilter=False):
    out_path = in_path.parent / (f'subsampled_{len(wl_bins)}_' + in_path.stem.split('.PHOENIX')[0] + '.fits')
    if out_path.is_file():
        if not overwrite:
            return (out_path, 'exists')
    try:
        msg = ''
        with fits.open(in_path) as f:
            if 'BUNIT' in f[0].header:
                unit = u.Unit(f[0].header['BUNIT'])
            else:
                unit = u.erg/u.s * u.cm**-2 / u.cm
                
            model_spec = Spectrum1D(spectral_axis=phoenix_wave, flux=f[0].data << unit)

        if prefilter:
            msk = (wl_bins[0] <= model_spec.spectral_axis) & (model_spec.spectral_axis<=wl_bins[-1])
            sub_spec = Spectrum1D(spectral_axis=model_spec.spectral_axis[msk], 
                                                    flux=model_spec.flux[msk])
        else:
            sub_spec = Spectrum1D(spectral_axis=model_spec.spectral_axis, 
                                                    flux=model_spec.flux)
            
        
        resampled = resampler.resample1d(sub_spec, wl_mid)
        resampled.write(out_path, format='tabular-fits', overwrite=True)
    
        return (out_path, msg) # means success
    except Exception as e:
        return (None, str(e)) # failed

In [14]:
pool = multiprocess.Pool()
res = list(tqdm(pool.imap(lambda x:do_resample(x, overwrite=False, prefilter=False), (m[0] for m in matches)), total=len(matches)))

failures = [(msg, matches[i][0]) for i, (fn, msg) in enumerate(res) if fn is None]
assert len(failures)<1, str(failures)

[(msg, matches[i][0]) for i, (fn, msg) in enumerate(res) if msg != '' and msg != 'exists']

  0%|          | 0/7559 [00:00<?, ?it/s]

[]

In [15]:
subsampled_matches = [PhoenixFiles(r[0], *m[1:]) for r, m in zip(res, matches)]

In [16]:
with fits.open(subsampled_matches[0].pth) as f:
    d = f[1].data['flux']
    wl = f[1].data['wavelength']
d.size*d.dtype.itemsize, wl.size*wl.dtype.itemsize

(382608, 382608)

We really only need the wl once, it's already above. How much memory will this take?

In [17]:
len(subsampled_matches)*d.size*d.dtype.itemsize/1024/1024 #MB

2758.153793334961

Ah so not a problem at all to keep in memory at this size.

In [ ]:
fluxes = []
for sm in tqdm(subsampled_matches):
    spec = Spectrum1D.read(sm.pth)
    fluxes.append(spec.flux)
fluxes = u.Quantity(fluxes)
fluxes.shape

  0%|          | 0/7559 [00:00<?, ?it/s]

In [ ]:
perspec_features = {}
for fnm in sm._fields[1:]:
    flst = []
    for sm in subsampled_matches:
        flst.append(getattr(sm, fnm))
    perspec_features[fnm + 's'] = np.array(flst, dtype=float)
perspec_features.keys()

We will also need photometry estimates. Check out [proof_of_concept_specphot](proof_of_concept_specphot.ipynb) for that.